In [1]:
from os.path import join
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

DATA_FOLDER='data_templates/'
POKEMON_FILE=join(DATA_FOLDER,'pokemon.csv')
COMBATS_FILE=join(DATA_FOLDER,'combats.csv')

df_pokemons=pd.read_csv(POKEMON_FILE,index_col=0)
df_combats=pd.read_csv(COMBATS_FILE)

# Compare means (ttest)

In [2]:
grass_class=((df_pokemons['Class 1']=='Grass') | (df_pokemons['Class 2']=='Grass'))
rock_class=((df_pokemons['Class 1']=='Rock') | (df_pokemons['Class 2']=='Rock'))

only_grass=df_pokemons[grass_class & np.logical_not(rock_class)]
only_rock=df_pokemons[rock_class & np.logical_not(grass_class)]

print('Grass attack Mean=%d and Variance=%d' % (only_grass.Attack.mean(),only_grass.Attack.std()))
print('Rock attack Mean=%d and Variance=%d'% (only_rock.Attack.mean(),only_rock.Attack.std()))

Grass attack Mean=73 and Variance=24
Rock attack Mean=91 and Variance=34


In [8]:
from scipy.stats import ttest_ind
#If variances are equal set equal_var=True
ttest_ind(only_rock.Attack,only_grass.Attack,equal_var=False,alternative='greater')

Ttest_indResult(statistic=3.4169379582521513, pvalue=0.0004820273456772652)

# Bootstrap

### Bootstrap CI func

In [5]:
def bootstrap_CI_func(data,func, nbr_draws=1000):
    res = np.zeros(nbr_draws)
    data = np.array(data)

    for n in range(nbr_draws):
        indices = np.random.randint(0, len(data), len(data))
        data_tmp = data[indices] 
        res[n] = func(data_tmp)

    return [np.nanpercentile(res, 2.5),np.nanpercentile(res, 97.5)]

def bootstrap_CI_values(data, nbr_draws=1000):
    bs_array=np.zeros(nbr_draws)
    for i in range(1000):
        index=np.random.randint(0, len(data), 1)
        bs_sample = data[index] 
        bs_array[i]=bs_sample
        
    return [np.nanpercentile(bs_array, 2.5),np.nanpercentile(bs_array, 97.5)]

In [8]:
print('Mean CI', bootstrap_CI_func(only_grass.Attack,np.mean))
print('Value CI', bootstrap_CI_values(only_grass.Attack.values))

Mean CI [68.99892473118278, 78.56075268817204]
Value CI [30.0, 110.0]


# Train_test_split

In [9]:
def train_test_split(df,p=0.8):
    shuffled_df=df.iloc[np.random.permutation(len(df))]
    
    Train=shuffled_df.iloc[:int(p*len(df))]
    Test=shuffled_df.iloc[int(p*len(df)):]
    
    return Train,Test